In [4]:
import math
import os
import random
import sys
import time
import re

import editdistance as ed
import numpy as np

Fqu = []
Flo = []
Ffi = []

with open('../data_archive/0420/rand_train3.fi') as f_fi:
    with open('../data_archive/0420/rand_train3.qu') as f_qu:
        with open('../data_archive/0420/rand_train3.lo') as f_lo:
            schema, query, logic = f_fi.readline(), f_qu.readline(), f_lo.readline()
            idx = 0
            while schema and query and logic:
                Fqu.append(query)
                Flo.append(logic)
                Ffi.append(schema)
                schema, query, logic = f_fi.readline(), f_qu.readline(), f_lo.readline()
                
num = np.random.permutation(len(Fqu))
print num
f_fi1 = open('../data_archive/0420/rand_train1.fi', 'w')
f_lo1 = open('../data_archive/0420/rand_train1.lo', 'w')
f_qu1 = open('../data_archive/0420/rand_train1.qu', 'w')

for i in range(len(num)):
    f_lo1.write(Flo[num[i]])
    f_fi1.write(Ffi[num[i]])
    f_qu1.write(Fqu[num[i]])

f_fi1.close()
f_qu1.close()
f_lo1.close()

[1787  876  286 ...,  952  724 3049]


In [11]:
import math
import os
import random
import sys
import time
import re

import numpy as np
import tensorflow as tf
import data_utils_tag

_buckets = [(10, 8), (15, 12), (19, 16), (23, 21)]  # _buckets = [(11, 8), (15, 12), (20, 16), (24, 21)]
max_size = None
data_set = [[] for _ in _buckets]

with tf.gfile.GFile('../data/rand_train.qu', mode="r") as source_file:
    with tf.gfile.GFile('../data/rand_train.lo', mode="r") as target_file:
        with tf.gfile.GFile('../data/rand_train.ta', mode="r") as tag_file:
            source, target, tag = source_file.readline(), target_file.readline(), tag_file.readline()
            counter = 0
            while source and target and tag and (not max_size or counter < max_size):
                counter += 1
                if counter % 200 == 0:
                    print("  reading data line %d" % counter)
                    sys.stdout.flush()
                source_ids = [x for x in source.split()]
                target_ids = [x for x in target.split()]
                tag_ids = [x for x in tag.split()]
                target_ids.append(data_utils_tag.EOS_ID)
                for bucket_id, (source_size, target_size) in enumerate(_buckets):
                    if len(source_ids) < source_size and len(target_ids) < target_size:
                        data_set[bucket_id].append([source_ids, tag_ids, target_ids])
                        break
                source, target, tag = source_file.readline(), target_file.readline(), tag_file.readline()

print "bucket 0: %d" % len(data_set[0])
print "bucket 1: %d" % len(data_set[1])
print "bucket 2: %d" % len(data_set[2])
print "bucket 3: %d" % len(data_set[3])



  reading data line 200
  reading data line 400
  reading data line 600
  reading data line 800
  reading data line 1000
  reading data line 1200
  reading data line 1400
  reading data line 1600
  reading data line 1800
  reading data line 2000
  reading data line 2200
  reading data line 2400
  reading data line 2600
  reading data line 2800
bucket 0: 801
bucket 1: 732
bucket 2: 711
bucket 3: 556
